# How to use this notebook

This notebook is meant to be run in a VertexAI Workbench. Here's how to start a VertexAI Workbench notebook:
https://bluestate.atlassian.net/wiki/spaces/ATeam/pages/2983624708/How+to+start+a+VertexAI+Workbench

You'll need to clone this notebook into VertexAI. Here's how to clone in Vertex AI:
https://cloud.google.com/vertex-ai/docs/workbench/user-managed/save-to-github#:~:text=In%20your%20GitHub%20repository%2C%20click,Click%20the%20Git%20clone%20button.

Finally, you'll need to grant this project the right to access `bsd-stitch`. Here's how to do that:
https://bluestate.atlassian.net/wiki/spaces/ATeam/pages/2983624741/Access+Stitch+data+from+another+project

Now you should be able to run this notebook successfully from VertexAI!

In [ ]:
import os

PROJECT_ID = "[your-project-id]" 
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
search_statement = PROJECT_ID
if search_statement.startswith('bsd-'):
    remove_bsd=input('Your project_id starts with bsd-, can I remove that to find schemas that contain only the client code? (only accepts "yes")')
    if remove_bsd=='yes':
                     search_statement=search_statement[4:]

In [ ]:
print(search_statement)

In [ ]:
from time import sleep
from google.cloud import bigquery

# Construct a BigQuery client object.
stitch_client = bigquery.Client('bsd-stitch')
current_project_client = bigquery.Client()

if datasets := list(stitch_client.list_datasets()):
    for dataset in datasets:
        dataset_id = dataset.dataset_id
        if search_statement in dataset_id:
            copy_dataset = input(f'Should I create a set of views based on {dataset_id}? (only accepts "yes")')
            if copy_dataset == 'yes':
                project_dataset = f"{PROJECT_ID}.src_stitch_{dataset_id}"
                dataset = bigquery.Dataset(project_dataset)

                # TODO(developer): Specify the geographic location where the dataset should reside.
                dataset.location = "US"

                # Send the dataset to the API for creation, with an explicit timeout.
                # Raises google.api_core.exceptions.Conflict if the Dataset already
                # exists within the project.
                try:
                    dataset = current_project_client.create_dataset(dataset, timeout=30)
                except Exception:
                    print("dataset already exists")
                tables = stitch_client.list_tables(dataset_id)
                qs=[]
                for table in tables:
                    clone_table_statement=f"""
                        CREATE OR REPLACE VIEW
                          {project_dataset}.{table.table_id}
                          AS SELECT * FROM bsd-stitch.{dataset_id}.{table.table_id}
                        """
                    qs.append(current_project_client.query(clone_table_statement))
                    print(f"Started cloning to bsd-stitch.{dataset_id}.{table.table_id}\n")
                num_queries = len(qs)
                num_complete = sum(q.running()==False for q in qs)
                while num_complete < num_queries:
                    print(f"{num_complete} out of {num_queries} completed!")
                    sleep(10)
                    num_complete = sum(q.running()==False for q in qs)
                print(f"\nFinished {dataset_id}!")
        